## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import all_estimators
from sklearn.base import RegressorMixin

## Define the files to be loaded

In [2]:
# Participant number whose data should be loaded
participant_number = 1

# The version of the questionnaires to load
quest_version = 2

## Loading the pre-processed data

In [3]:
df = pd.read_csv('data/preprocessed/preprocessed_data_' + str(participant_number) + '_v' + str(quest_version) + '.csv')
df = df.drop(labels=['actual_day', 'actual_day.1'], axis=1)

df_oura = pd.read_csv('data/preprocessed/preprocessed_sleep_' + str(participant_number) + '_v' + str(quest_version) + '.csv')
df_oura = df_oura.drop(labels=['actual_day'], axis=1)

df_quest = pd.read_csv('data/preprocessed/preprocessed_questionnaires_' + str(participant_number) + '_v' + str(quest_version) + '.csv')
df_quest = df_quest.drop(labels=['actual_day'], axis=1)

In [4]:
# Remove these to try and fix the issues raised by LazyRegressor for them 
# example: 
# AdaBoostRegressor model failed to execute
# Found unknown categories ['[0.0, 51.0, 53.0, 53.0, 54.0, 53.0, ...
df = df.drop(labels=['hrv.items', 'heart_rate.items', 'sleep_phase_5_min', 'movement_30_sec'], axis=1)
df_oura = df_oura.drop(labels=['hrv.items', 'heart_rate.items', 'sleep_phase_5_min', 'movement_30_sec'], axis=1)

In [5]:
print(df.columns.tolist())

['average_breath', 'average_breath_variation', 'average_heart_rate', 'average_hrv', 'awake_time', 'deep_sleep_duration', 'efficiency', 'got_ups', 'latency', 'light_sleep_duration', 'lowest_heart_rate', 'lowest_heart_rate_time_offset', 'period', 'rem_sleep_duration', 'restless_periods', 'score', 'segment_state', 'sleep_midpoint', 'time_in_bed', 'total_sleep_duration', 'type', 'wake_ups', 'restless', 'timezone', 'bedtime_start_delta', 'bedtime_end_delta', 'contributors.total_sleep', 'contributors.deep_sleep', 'contributors.rem_sleep', 'contributors.efficiency', 'contributors.latency', 'contributors.restfulness', 'contributors.timing', 'readiness.contributors.activity_balance', 'readiness.contributors.body_temperature', 'readiness.contributors.hrv_balance', 'readiness.contributors.previous_day_activity', 'readiness.contributors.previous_night', 'readiness.contributors.recovery_index', 'readiness.contributors.resting_heart_rate', 'readiness.contributors.sleep_balance', 'readiness.score', '

## Prepare regressor list for lazypredict

In [6]:
removed_regressors = [
    "TheilSenRegressor",
    "ARDRegression", 
    "CCA", 
    "IsotonicRegression", 
    "StackingRegressor",
    "MultiOutputRegressor", 
    "MultiTaskElasticNet", 
    "MultiTaskElasticNetCV", 
    "MultiTaskLasso", 
    "MultiTaskLassoCV", 
    "PLSCanonical", 
    "PLSRegression", 
    "RadiusNeighborsRegressor", 
    "RegressorChain", 
    "VotingRegressor", 
]

In [7]:
REGRESSORS = [
    est
    for est in all_estimators()
    if (issubclass(est[1], RegressorMixin) and (est[0] not in removed_regressors))
]

## Run LazyRegressor on datasets

In [8]:
def lazy_regressor(X, y, test_size, random_state, regressor):
    # Create the test and train sets for the given prediction
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = random_state)
    
    models, predictions = regressor.fit(X_train, X_test, y_train, y_test)
    return models

In [9]:
# Apply the LazyRegressor on the data
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None, regressors = REGRESSORS)

### LazyRegressor for the factors influencing the sleep scores given all the variables

In [10]:
# Separate the indenpendent variables from the dependent variables
y_oura = df['score']
y_sub = df['subjective_sleep_score']
X = df.drop(labels=['score', 'subjective_sleep_score'], axis=1)

#### Oura sleep scores

In [11]:
models_oura = lazy_regressor(X, y_oura, test_size = 0.2, random_state = 2, regressor = reg)

print(models_oura)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 65%|██████▌   | 26/40 [00:00<00:00, 26.60it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


100%|██████████| 40/40 [00:01<00:00, 32.19it/s]

RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 44.
                                                              Adjusted R-Squared  \
Model                                                                              
Lars                          21230400555833760238810433898294115034118947340...   
GaussianProcessRegressor                                                   78.68   
MLPRegressor                                                                5.60   
KernelRidge                                                                 1.32   
LinearSVR                                                                   1.25   
HuberRegressor                                                              1.19   
QuantileRegressor                                                           1.17   
SGDRegressor                                                                1.16   
PassiveAggressiveRegressor                      

#### Subjective sleep scores

In [12]:
models_sub = lazy_regressor(X, y_sub, test_size = 0.2, random_state = 6, regressor = reg)

print(models_sub)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 65%|██████▌   | 26/40 [00:01<00:00, 20.92it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.
RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 44.


100%|██████████| 40/40 [00:01<00:00, 26.17it/s]

                                         Adjusted R-Squared  \
Model                                                         
Lars                          12309498013805496391172096.00   
GaussianProcessRegressor                              25.85   
LinearSVR                                              4.71   
HuberRegressor                                         4.49   
TransformedTargetRegressor                             4.42   
LinearRegression                                       4.42   
KernelRidge                                            4.07   
MLPRegressor                                           3.87   
PassiveAggressiveRegressor                             3.81   
Ridge                                                  3.50   
SGDRegressor                                           2.37   
PoissonRegressor                                       2.06   
ExtraTreeRegressor                                     1.92   
RidgeCV                                                

### LazyRegressor for the factors influencing the sleep scores given the questionnaire variables only

In [13]:
# Separate the indenpendent variables from the dependent variables
X = df_quest.drop(labels=['subjective_sleep_score'], axis=1)

#### Oura sleep score

In [14]:
models_oura = lazy_regressor(X, y_oura, test_size = 0.2, random_state = 2, regressor = reg)

print(models_oura)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 65%|██████▌   | 26/40 [00:01<00:00, 20.24it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


100%|██████████| 40/40 [00:01<00:00, 23.25it/s]

                                        Adjusted R-Squared  \
Model                                                        
RANSACRegressor               4323506577069073695244288.00   
GaussianProcessRegressor                            192.15   
Lars                                                104.33   
KernelRidge                                           7.93   
MLPRegressor                                          6.05   
SGDRegressor                                          2.76   
PassiveAggressiveRegressor                            2.72   
HuberRegressor                                        2.51   
LinearSVR                                             2.21   
Ridge                                                 1.97   
PoissonRegressor                                      1.94   
LinearRegression                                      1.92   
TransformedTargetRegressor                            1.92   
RidgeCV                                               1.80   
Decision

#### Subjective sleep scores

In [15]:
models_sub = lazy_regressor(X, y_sub, test_size = 0.2, random_state = 6, regressor = reg)

print(models_sub)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 65%|██████▌   | 26/40 [00:01<00:00, 18.03it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


100%|██████████| 40/40 [00:01<00:00, 22.31it/s]

                               Adjusted R-Squared  R-Squared  RMSE  Time Taken
Model                                                                         
Lars                                       212.88    -655.83 12.33        0.01
GaussianProcessRegressor                    61.98    -188.03  6.61        0.01
LinearRegression                             7.16     -18.10  2.10        0.01
TransformedTargetRegressor                   7.16     -18.10  2.10        0.01
KernelRidge                                  6.04     -14.63  1.90        0.01
MLPRegressor                                 5.90     -14.20  1.88        0.09
LinearSVR                                    5.04     -11.53  1.70        0.01
HuberRegressor                               4.67     -10.37  1.62        0.02
RANSACRegressor                              4.60     -10.15  1.61        0.07
SGDRegressor                                 3.98      -8.25  1.46        0.01
PassiveAggressiveRegressor                   2.91   

### LazyRegressor for the factors influencing the sleep scores given the Oura variables only

In [16]:
# Separate the indenpendent variables from the dependent variables
X = df_oura.drop(labels=['score'], axis=1)

#### Oura sleep score

In [17]:
models_oura = lazy_regressor(X, y_oura, test_size = 0.2, random_state = 2, regressor = reg)

print(models_oura)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 68%|██████▊   | 27/40 [00:00<00:00, 32.04it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


100%|██████████| 40/40 [00:01<00:00, 36.15it/s]

RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 44.
                               Adjusted R-Squared  R-Squared   RMSE  \
Model                                                                 
Lars                                     33004.25 -125411.35 130.30   
GaussianProcessRegressor                   158.40    -597.12   9.00   
MLPRegressor                                 4.71     -13.11   1.38   
QuantileRegressor                            1.35      -0.32   0.42   
DummyRegressor                               1.30      -0.16   0.40   
ElasticNet                                   1.30      -0.16   0.40   
Lasso                                        1.30      -0.16   0.40   
LassoLars                                    1.30      -0.16   0.40   
KNeighborsRegressor                          1.16       0.40   0.29   
ExtraTreeRegressor                           1.14       0.46   0.27   
HistGradientBoostingRegressor       

#### Subjective sleep scores

In [18]:
models_sub = lazy_regressor(X, y_sub, test_size = 0.2, random_state = 6, regressor = reg)

print(models_sub)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 65%|██████▌   | 26/40 [00:02<00:01, 12.08it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


100%|██████████| 40/40 [00:02<00:00, 15.65it/s]

RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 44.
                               Adjusted R-Squared  R-Squared  RMSE  Time Taken
Model                                                                         
Lars                                      1029.15   -3905.97 30.07        0.02
GaussianProcessRegressor                    51.34    -190.29  6.65        0.01
LinearRegression                            11.84     -40.21  3.09        0.01
TransformedTargetRegressor                  11.84     -40.21  3.09        0.01
HuberRegressor                              10.23     -34.06  2.85        0.03
LinearSVR                                    5.05     -14.40  1.89        0.01
MLPRegressor                                 4.88     -13.76  1.85        0.10
KernelRidge                                  2.99      -6.55  1.32        0.01
PassiveAggressiveRegressor                   2.92      -6.28  1.30        0.01
Ridge              

### LazyRegressor for the factors influencing the average HRV given all the variables

In [19]:
# Separate the indenpendent variables from the dependent variables
y_hrv = df['average_hrv']
X_hrv = df.drop(labels=['average_hrv'], axis=1)

In [20]:
models_hrv = lazy_regressor(X_hrv, y_hrv, test_size = 0.2, random_state = 5, regressor = reg)

print(models_hrv)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 65%|██████▌   | 26/40 [00:01<00:00, 24.52it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


100%|██████████| 40/40 [00:01<00:00, 30.11it/s]

RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 44.
                                                              Adjusted R-Squared  \
Model                                                                              
Lars                          50498121913902372371518224500303226187335954109...   
GaussianProcessRegressor                                                    6.94   
MLPRegressor                                                                1.65   
KernelRidge                                                                 1.27   
LinearSVR                                                                   1.21   
HuberRegressor                                                              1.21   
PassiveAggressiveRegressor                                                  1.20   
SGDRegressor                                                                1.16   
DummyRegressor                                  

### LazyRegressor for the factors influencing the average HRV given the questionnaire variables only

In [21]:
X_hrv = df_quest

In [22]:
models_hrv = lazy_regressor(X_hrv, y_hrv, test_size = 0.2, random_state = 5, regressor = reg)

print(models_hrv)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 65%|██████▌   | 26/40 [00:01<00:00, 15.53it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


100%|██████████| 40/40 [00:01<00:00, 21.85it/s]

RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 44.
                               Adjusted R-Squared  R-Squared  RMSE  Time Taken
Model                                                                         
GaussianProcessRegressor                    15.36     -44.95 43.67        0.01
MLPRegressor                                 5.60     -13.71 24.71        0.15
KernelRidge                                  5.13     -12.22 23.42        0.01
Lars                                         4.41      -9.91 21.28        0.02
PassiveAggressiveRegressor                   3.20      -6.05 17.11        0.01
LinearRegression                             2.90      -5.08 15.89        0.01
TransformedTargetRegressor                   2.90      -5.08 15.89        0.01
LinearSVR                                    2.50      -3.79 14.10        0.01
SGDRegressor                                 2.49      -3.77 14.07        0.01
HuberRegressor     

### LazyRegressor for the factors influencing the average HRV given the Oura variables only

In [23]:
X_hrv = df_oura.drop(labels=['average_hrv'], axis=1)

In [24]:
models_hrv = lazy_regressor(X_hrv, y_hrv, test_size = 0.2, random_state = 5, regressor = reg)

print(models_hrv)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 65%|██████▌   | 26/40 [00:01<00:00, 14.28it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


100%|██████████| 40/40 [00:01<00:00, 20.62it/s]

RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 44.
                               Adjusted R-Squared  R-Squared   RMSE  \
Model                                                                 
Lars                                       270.94   -1024.77 206.33   
GaussianProcessRegressor                    13.18     -45.30  43.84   
MLPRegressor                                 3.23      -7.47  18.75   
DecisionTreeRegressor                        1.28      -0.08   6.70   
DummyRegressor                               1.27      -0.04   6.56   
LassoLars                                    1.27      -0.04   6.56   
LinearRegression                             1.27      -0.01   6.47   
TransformedTargetRegressor                   1.27      -0.01   6.47   
QuantileRegressor                            1.26      -0.00   6.45   
HuberRegressor                               1.25       0.04   6.32   
NuSVR                               

### LazyRegressor for the factors influencing the total sleep duration given all variables

In [25]:
# Separate the indenpendent variables from the dependent variables
# For the total sleep duration, only consider the days for which a person woke up naturally
y_total_sleep = df[df['Naturally'] == 1]['total_sleep_duration']
X_total_sleep = df[df['Naturally'] == 1]
X_total_sleep = X_total_sleep.drop(labels=['total_sleep_duration', 'deep_sleep_duration', 'light_sleep_duration', 'contributors.total_sleep', 'contributors.deep_sleep', 'contributors.rem_sleep', 'rem_sleep_duration', 'time_in_bed', 'bedtime_start_delta', 'bedtime_end_delta', 'bedtime_end_seconds', 'bed_time'], axis=1)

In [26]:
models_sleep_duration = lazy_regressor(X_total_sleep, y_total_sleep, test_size = 0.2, random_state = 89, regressor = reg)

print(models_sleep_duration)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 65%|██████▌   | 26/40 [00:00<00:00, 27.48it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


 90%|█████████ | 36/40 [00:01<00:00, 31.20it/s]

RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 37.


100%|██████████| 40/40 [00:01<00:00, 32.13it/s]

                                                              Adjusted R-Squared  \
Model                                                                              
Lars                          36179072918332640373649557110042654615147642880...   
GaussianProcessRegressor                                                   17.59   
MLPRegressor                                                               17.55   
LinearSVR                                                                  17.44   
ExtraTreeRegressor                                                          1.19   
DecisionTreeRegressor                                                       1.15   
DummyRegressor                                                              1.14   
HistGradientBoostingRegressor                                               1.14   
QuantileRegressor                                                           1.13   
SVR                                                                         

### LazyRegressor for the factors influencing the total sleep duration given the questionnaire variables only

In [27]:
X_total_sleep = df_quest[df_quest['Naturally'] == 1]

In [28]:
models_sleep_duration = lazy_regressor(X_total_sleep, y_total_sleep, test_size = 0.2, random_state = 89, regressor = reg)

print(models_sleep_duration)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 65%|██████▌   | 26/40 [00:01<00:00, 21.38it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


100%|██████████| 40/40 [00:01<00:00, 29.44it/s]

RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 37.
                               Adjusted R-Squared  R-Squared      RMSE  \
Model                                                                    
Lars                                     35301.88 -129435.55 953265.17   
MLPRegressor                                35.14    -124.18  29645.58   
LinearSVR                                   35.05    -123.84  29604.39   
GaussianProcessRegressor                    34.93    -123.42  29555.20   
PoissonRegressor                            12.26     -40.29  17025.57   
TransformedTargetRegressor                   6.75     -20.07  12162.50   
LinearRegression                             6.75     -20.07  12162.50   
LassoLars                                    6.39     -18.76  11777.43   
Lasso                                        5.89     -16.91  11214.81   
KernelRidge                                  4.53     -11.93   9526.90   


### LazyRegressor for the factors influencing the total sleep duration given the Oura variables only

In [29]:
X_total_sleep = df[df['Naturally'] == 1]
X_total_sleep = X_total_sleep.iloc[:, 0:49]
X_total_sleep = X_total_sleep.drop(labels=['total_sleep_duration', 'deep_sleep_duration', 'light_sleep_duration', 'contributors.total_sleep', 'contributors.deep_sleep', 'contributors.rem_sleep', 'rem_sleep_duration', 'time_in_bed', 'bedtime_start_delta', 'bedtime_end_delta', 'bedtime_end_seconds', 'bed_time'], axis=1)

In [30]:
models_sleep_duration = lazy_regressor(X_total_sleep, y_total_sleep, test_size = 0.2, random_state = 89, regressor = reg)

print(models_sleep_duration)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 70%|███████   | 28/40 [00:01<00:00, 25.48it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


100%|██████████| 40/40 [00:01<00:00, 31.56it/s]

RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 37.
                               Adjusted R-Squared  R-Squared      RMSE  \
Model                                                                    
Lars                                      8748.67  -27213.98 437108.50   
GaussianProcessRegressor                    41.29    -124.36  29666.34   
MLPRegressor                                41.21    -124.09  29634.32   
LinearSVR                                   41.00    -123.45  29558.00   
TransformedTargetRegressor                   1.88      -1.75   4391.27   
LinearRegression                             1.88      -1.75   4391.27   
Lasso                                        1.48      -0.49   3238.54   
DecisionTreeRegressor                        1.37      -0.14   2827.20   
PoissonRegressor                             1.36      -0.11   2791.49   
HistGradientBoostingRegressor                1.33      -0.03   2684.12   


### LazyRegressor for the factors influencing the total awake time during the night given all variables

In [31]:
# Separate the indenpendent variables from the dependent variables
# For the total awake time, Oura also considers some time before falling asleep and some time after waking up. 
# Subtract what we can from that, which is the time before falling asleep
y_awake_time = df['awake_time'] - df['latency']
X_awake_time = df.drop(labels=['awake_time', 'time_in_bed', 'latency', 'total_sleep_duration'], axis=1)

In [32]:
models_awake_time = lazy_regressor(X_awake_time, y_awake_time, test_size = 0.2, random_state = 56, regressor = reg)

print(models_awake_time)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 65%|██████▌   | 26/40 [00:01<00:00, 21.26it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


100%|██████████| 40/40 [00:01<00:00, 29.13it/s]

RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 44.
                                                              Adjusted R-Squared  \
Model                                                                              
Lars                          17319134925288230532342993281005408603266436667...   
PoissonRegressor                                                            4.00   
GaussianProcessRegressor                                                    2.61   
MLPRegressor                                                                2.57   
LinearSVR                                                                   2.50   
NuSVR                                                                       1.22   
DummyRegressor                                                              1.18   
QuantileRegressor                                                           1.15   
SVR                                             

### LazyRegressor for the factors influencing the total awake time during the night given the questionnaire variables only

In [33]:
X_awake_time = df_quest

In [34]:
models_awake_time = lazy_regressor(X_awake_time, y_awake_time, test_size = 0.2, random_state = 56, regressor = reg)

print(models_awake_time)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 65%|██████▌   | 26/40 [00:00<00:00, 25.86it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


100%|██████████| 40/40 [00:01<00:00, 32.72it/s]

RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 44.
                               Adjusted R-Squared  R-Squared     RMSE  \
Model                                                                   
Lars                                       731.52   -2336.67 67583.65   
MLPRegressor                                 4.72     -10.91  4824.81   
GaussianProcessRegressor                     4.72     -10.91  4823.79   
LinearSVR                                    4.66     -10.70  4780.93   
PoissonRegressor                             2.31      -3.19  2859.67   
TransformedTargetRegressor                   1.82      -1.63  2264.93   
LinearRegression                             1.82      -1.63  2264.93   
Lasso                                        1.72      -1.30  2120.61   
NuSVR                                        1.53      -0.68  1814.36   
SGDRegressor                                 1.43      -0.36  1631.23   
DummyRegress

### LazyRegressor for the factors influencing the total awake time during the night given the Oura variables only

In [35]:
X_awake_time = df_oura.drop(labels=['awake_time', 'time_in_bed', 'latency', 'total_sleep_duration'], axis=1)

In [36]:
models_awake_time = lazy_regressor(X_awake_time, y_awake_time, test_size = 0.2, random_state = 56, regressor = reg)

print(models_awake_time)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


 65%|██████▌   | 26/40 [00:00<00:00, 29.28it/s]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.
RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 44.


100%|██████████| 40/40 [00:01<00:00, 34.64it/s]

                               Adjusted R-Squared  R-Squared     RMSE  \
Model                                                                   
Lars                                       688.15   -2404.03 68550.51   
GaussianProcessRegressor                     4.45     -11.08  4858.07   
MLPRegressor                                 4.38     -10.81  4804.68   
LinearSVR                                    4.26     -10.42  4723.71   
PoissonRegressor                             2.30      -3.54  2978.64   
NuSVR                                        1.48      -0.68  1814.43   
DummyRegressor                               1.38      -0.33  1609.77   
QuantileRegressor                            1.33      -0.17  1509.35   
SVR                                          1.33      -0.16  1503.87   
TransformedTargetRegressor                   1.23       0.19  1261.14   
LinearRegression                             1.23       0.19  1261.14   
ExtraTreeRegressor                           1.17  